## Movielens clustering

In this Notebook, we are looking for user clusters in the Movielens data, using _k_-means clustering.

In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans #The k-means algorithm

First, let's start by creating a user-item matrix, as explained in the other Notebook.

In [17]:
movie_file = pd.read_csv('movies.csv')
ratings_file = pd.read_csv('ratings.csv')
df = pd.merge(movie_file, ratings_file)

ratings = pd.pivot_table(df, index='userId', columns='title', values='rating')
ratings.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's put the most popular movies at the front.

In [66]:
#This piece of code is a bit complex. Here it is, step by step:
#1. reindex shuffles a dataframe according to a new list
#2. ratings.count() gets the number of non-NaN values per column/movie
#3. sort_values() sort those values, descending (because ascending=False)
#4. finally, .index gets the names of the columns/movies
#axis=1 tells Pandas we want to reshuffle the columns (not the rows)
ratings = ratings.reindex(ratings.count().sort_values(ascending=False).index, axis=1)
ratings.head(3)

title,cluster,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),...,"Last Song, The (2010)",Last Train Home (2009),"Last Waltz, The (1978)","Last Wave, The (1977)","Last Wedding, The (Kivenpyörittäjän kylä) (1995)","Last Winter, The (2006)",Last Year's Snow Was Falling (1983),Last of the Dogmen (1995),Late Marriage (Hatuna Meuheret) (2001),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will now find clusters. Unfortunately, the _k_-means algorithm won't work with NaN values. We will put a 0 in the empty cells. This is not ideal for many reasons, but the best we can do for now without getting really complex

In [67]:
ratings_full = ratings.fillna(0) #fill the NaN with the mean of each column
ratings_full.head(3)

title,cluster,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),...,"Last Song, The (2010)",Last Train Home (2009),"Last Waltz, The (1978)","Last Wave, The (1977)","Last Wedding, The (Kivenpyörittäjän kylä) (1995)","Last Winter, The (2006)",Last Year's Snow Was Falling (1983),Last of the Dogmen (1995),Late Marriage (Hatuna Meuheret) (2001),'71 (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0,4.0,0.0,3.0,4.0,5.0,5.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, let's try clustering the users. We first need to figure out the number of clusters. Something like 7 clusters seems like a useful number to start: big enough to capture differences between users, but small enough to be manageable and understandable.

This is really just an educated guess. There are more [sophisticated ways to do it](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py), but even they are a bit arbitrary. 

In [71]:
km = KMeans(n_clusters=7, random_state=1) #create a new k-means model with 7 clusters. I've used random_state=1 to get stable output
km = km.fit(ratings_full) #calculate the cluster centers
ratings['cluster'] = km.predict(ratings_full) #predict the clusters of each observation and store in the dataframe


Let's see how many of each cluster there are:

In [82]:
ratings['cluster'].value_counts() 

6    343
3    105
1     98
4     41
0     18
2      3
5      2
Name: cluster, dtype: int64

It seems like there are only four big clusters (>40), and three smaller ones. Let's get the values of the movies for each cluster using a pivot table:

In [85]:
mean_ratings = pd.pivot_table(ratings, index='cluster')
mean_ratings = mean_ratings.reindex(ratings.count().sort_values(ascending=False).index, axis=1)
mean_ratings.iloc[[6,3,1,4],1:15] #I will only show the clusters with a substantial number of members, and show the 15 most popular movies

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),Fight Club (1999),Toy Story (1995),Star Wars: Episode V - The Empire Strikes Back (1980),"Usual Suspects, The (1995)",American Beauty (1999)
cluster,,,,,,,,,,,,,,,
6,4.102679,4.392562,4.193878,4.000000,4.156250,4.128713,3.554545,3.861538,3.953704,4.091463,4.191489,3.898551,3.971831,4.227273,3.933333
3,4.158537,4.535211,4.257143,4.115385,4.149425,4.207792,3.701754,3.855769,3.891667,4.240741,4.264286,3.947368,4.276316,4.322917,4.040323
1,4.290541,4.405405,3.901235,4.317460,4.333333,4.708333,3.949275,4.283784,3.966102,4.430233,NaN,3.986486,4.416667,4.089744,NaN
4,4.137500,4.364865,4.565789,4.314286,4.312500,4.282051,3.697368,4.033333,3.942857,4.250000,4.435897,3.712121,4.381579,4.290323,4.218750


Though this is typically hard, we can see some patterns in the movies:

6: The average user. In the middle for most of the movies.

3: No clear pattern, also mostly in the middle.

1: Likes action/blockbuster movies like The Matrix, Jurassic Park, Braveheart. Really likes Star Wars.

4: Seems to be a bit more into movies that are a bit more arthouse, like Pulp Fiction, Fight Club and American Beauty?